# Part 1: Building the MVP AI System  

In this first part, we’ll build a **zero-shot classification system** using a local **LLM-powered approach**. Instead of training a model from scratch, we’ll leverage **Gemma 2B**’s in-context learning to classify sports teams as **from the US or Australia**—without any fine-tuning.  

But an AI system is **more than just a model**. We'll also integrate:  
- **Gradio** to build an interactive front end.  
- **SQLite** to store data and results.  
- **Datasette** for observability, allowing us to inspect predictions and iterate effectively.  

By the end of this section, you’ll have a **working MVP AI system**—a functional app with a front end, database, and structured observability to track and refine performance.  


## Getting Started

In [9]:
from ollama import chat
from ollama import ChatResponse

model = 'gemma:2b'

def single_turn(prompt):
    response: ChatResponse = chat(model=model, messages=[
      {
        'role': 'user',
        'content': prompt,
      },
    ])
    return response['message']['content']

prompt = "Say hello to the class"
single_turn(prompt)

"Hello! It's great to be here with you all. How can I help you today?"

Let's try our zero-shot classification task!

In [10]:
afl_team = "Carlton Blues"
american_team = "Tennessee Titans"


In [11]:
prompt = "Output if this is an australian or american team, only print australian or american no other output: " + f"{afl_team}"
single_turn(prompt)

'The text is about an Australian team, so it is an Australian team and the output is "Australian".'

In [12]:
prompt = "Output if this is an australian or american team, only print australian or american no other output: " + f"{american_team}"
single_turn(prompt)

"The team's origin is not specified in the context, so I cannot determine if it is an Australian or American team."

## Creating our app

We'll now create our Gradio app. For instruction purposes, we've included the code below, but we'll be running our apps from the command line:

```
import gradio as gr
import ollama

model = 'gemma:2b'

def chat_with_model(prompt):
    response = ollama.chat(model=model, messages=[{'role': 'user', 'content': prompt}])
    return response['message']['content']

iface = gr.Interface(
    fn=chat_with_model,
    inputs=gr.Textbox(lines=2, placeholder="Type your message here..."),
    outputs="text",
    title="Chat with Gemma",
    description="Enter a message and get a response from the Gemma 2B model.",
)

iface.launch()
```

## Adding observability with SQLite and Datasette

We'll now log our traces (prompt + response) to a local (SQLite) database, in order to have consistent visbility into our system. As above, we've included the code below, although we'll be running our apps from the command line:

```
import gradio as gr
import ollama
import sqlite3
import datetime

# SQLite Database Setup
DB_PATH = "chat_log.db"

def setup_database():
    """Create a simple SQLite table if it doesn't exist."""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            prompt TEXT,
            response TEXT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    conn.close()

setup_database()  # Ensure the DB is set up before running the app

def chat_with_model(prompt):
    """Send user input to Ollama, get response, and log to SQLite."""
    response = ollama.chat(model="gemma:2b", messages=[{"role": "user", "content": prompt}])["message"]["content"]
    
    # Log the interaction to SQLite
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO chat_history (prompt, response) VALUES (?, ?)", (prompt, response))
    conn.commit()
    conn.close()

    return response

# Gradio UI
iface = gr.Interface(
    fn=chat_with_model,
    inputs=gr.Textbox(lines=2, placeholder="Type your message here..."),
    outputs="text",
    title="Chat with Gemma",
    description="Enter a message and get a response from the Gemma 2B model. Your chats are logged in SQLite.",
)

iface.launch()
```

There are lots of ways to look at your data in your SQLite database. We'll be using Simon Willison's `Datasette` tool.